In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [13]:
def scrape_products_from_page(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all product name elements
    product_elements = soup.find_all('div', class_='product-item-info')
    #print(product_elements[0].prettify())
    
    products=[]
    # Extract and print the names
    
    for product in product_elements[:-1]:
        try:
            # Product Link
            link_tag = product.find("a", class_="product-item-link")
            product_link = link_tag['href'] if link_tag else None
            
            
            # Product Full Name
            full_name = link_tag.text.strip() if link_tag else None
        
            # Image URL
            img_tag = product.find("img", class_="product-image-photo")
            image_url = img_tag['data-src'] if img_tag else None
            
            # Price
            price_tag = product.find("span", class_="price")
            price = price_tag.text.strip() if price_tag else None
            
            specs = {}
            #Additional data from product page
            if product_link:
                response = requests.get(product_link)
                if response.status_code == 200:                    
                    product_page = BeautifulSoup(response.text, 'html.parser')
                    table = product_page.find('table')
                    for row in table.find_all('tr'):
                            cells = row.find_all(['td', 'th'])
                            if len(cells) >= 2:
                                key = cells[0].get_text(strip=True)
                                value = cells[1].get_text(strip=True)
                                specs[key] = value

            
            # Append product data
            product_dict = {
                "lien": product_link,
                "nom": full_name,
                "image_url": image_url,
                "prix": price,
                "boutique":"graiet"
            }

            # Add specs dictionary to product_data (expand keys dynamically)
            product_dict.update(specs)

            # Append to products list
            products.append(product_dict)
            

        except Exception as e:
            print(f"Error processing product: {e}")
    
    return products

In [14]:
base_url = "https://www.graiet.tn/informatique-et-gaming/pc-portable.html?p={}"
all_products = []

for page_num in range(1, 15):
    print(f"Scraping page {page_num}...")
    url = base_url.format(page_num)
    page_products = scrape_products_from_page(url)
    all_products.extend(page_products)
    time.sleep(1) #A small delay between requests to avoid hammering the server.

print(f"Total products scraped: {len(all_products)}")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Total products scraped: 132


In [15]:
list(all_products[0].keys())

['lien',
 'nom',
 'image_url',
 'prix',
 'boutique',
 'Prix',
 'Référence',
 'Garantie',
 "Type d'ordinateur",
 'Taille Ecran',
 'Type Ecran',
 'Résolution',
 'Processeur',
 'Système D’exploitation',
 'Mémoire RAM',
 'Disque Dur',
 'Carte Graphique',
 'Bluetooth',
 'Ecran Tactile',
 'Pc Gamer',
 'Couleur']

In [16]:
columns_to_keep = ['Référence', 'lien', 'nom', 'Marque', 'image_url', 'boutique', 'Taille Ecran',
                   'Résolution', 'Processeur', 'Référence processeur', 'Mémoire RAM','Disque Dur', 
                   'Type de disque dur', 'Carte Graphique', "Système D’exploitation",'Pc Gamer', 'prix']

# Filter the dictionary to keep only the desired columns
filtered_product_features = [{key: product.get(key) for key in columns_to_keep}
                             for product in all_products]

In [17]:
df = pd.DataFrame(filtered_product_features)
df.head()

,Référence,lien,nom,Marque,image_url,boutique,Taille Ecran,Résolution,Processeur,Référence processeur,Mémoire RAM,Disque Dur,Type de disque dur,Carte Graphique,Système D’exploitation,Pc Gamer,prix
0,82QY00PEFE,https://www.graiet.tn/pc-portable-lenovo-v15-c...,Pc Portable LENOVO V15 | Intel Celeron N4500 -...,None,https://www.graiet.tn/media/catalog/product/ca...,graiet,15.6 “,Full HD 1080p,Hélio G99,None,8 G,256 Go SSD,None,Intel HD Graphics,Android 14,Non,"729,00 TND"
1,82VG00Q2FG-2Y,https://www.graiet.tn/lenovo-pc-portable-ideap...,Pc Portable LENOVO IDEAPAD 1 | AMD ATHLON - 7...,None,https://www.graiet.tn/media/catalog/product/ca...,graiet,15.6 “,Full HD 1080p,Hélio G99,None,8 G,256 Go SSD,None,AMD Radeon ™ Intégrée,FreeDos,Non,"765,00 TND"
2,886L5EA,https://www.graiet.tn/pc-portable-hp-15-fd0030...,Pc Portable HP 15 | Intel N100 - 4 GO - 256 SS...,None,https://www.graiet.tn/media/catalog/product/ca...,graiet,15.6 “,Full HD 1080p,Celeron N100,None,4 G,256 Go SSD,None,Intel UHD Graphics,Windows 11,Non,"779,00 TND"
3,AB1T8EA,https://www.graiet.tn/hp-pc-portable-15-ab1t8e...,Pc Portable HP 15 | Intel Celeron N100 - 8 Go ...,None,https://www.graiet.tn/media/catalog/product/ca...,graiet,15.6 “,(1600 x 720 ) Pixels,Celeron N100,None,8 G,256 Go SSD,None,Intel UHD Graphics,FreeDos,Non,"789,00 TND"
4,None,https://www.graiet.tn/hp-pc-portable-15-dw1002...,HP Pc Portable 15-DW1002NK (CELERON N4020/4G/1...,None,https://www.graiet.tn/media/catalog/product/ca...,graiet,15.6 “,(1366 x 768) Pixels,Celeron N4020,None,4 G,1 To,None,Intel HD Graphics,Windows 10,Non,"999,00 TND"


In [18]:
df.describe().T

,count,unique,top,freq
Référence,128,126,9S7-16R831-2607,2
lien,132,132,https://www.graiet.tn/pc-portable-lenovo-v15-c...,1
nom,132,128,Pc Portable HP 15 | Intel i5 - 16 Go - 512 SSD...,2
Marque,0,0,NaN,NaN
image_url,132,132,https://www.graiet.tn/media/catalog/product/ca...,1
boutique,132,1,graiet,132
Taille Ecran,131,6,15.6 “,118
Résolution,131,7,Full HD 1080p,117
Processeur,129,21,Intel Core i5 12è Gén,24
Référence processeur,0,0,NaN,NaN


In [19]:
# Save to CSV
df.to_csv("graiet_products.csv", index=False, encoding='utf-8-sig')

print("Data saved to graiet_products.csv")

Data saved to graiet_products.csv
